In [518]:
import pandas as pd

pd.set_option('display.max_columns', None)
df = pd.read_csv('games.csv').fillna(0)
df = df.sort_values(['game_date'])

In [519]:
# CSV preprocess for training
home = df[df['home_away'] == "home"].copy()
away = df[df['home_away'] == "away"].copy()

home.columns = [f"{col}_home" if col not in ['game_id', 'game_date'] else col 
               for col in home.columns]

away.columns = [f"{col}_away" if col not in ['game_id', 'game_date'] else col 
               for col in away.columns]

premerged = pd.merge(home, away, on=['game_id', 'game_date'])

merged = pd.DataFrame()
merged['game_id'] = premerged['game_id']
merged['game_date'] = premerged['game_date']
merged['team_home'] = premerged['team_home']
merged['team_away'] = premerged['team_away']
merged['home_away_NS'] = premerged['home_away_NS_home']
merged['rest_days_home'] = premerged['rest_days_home']
merged['rest_days_away'] = premerged['rest_days_away']
merged['travel_dist_home'] = premerged['travel_dist_home']
merged['travel_dist_away'] = premerged['travel_dist_away']
merged['team_score_home'] = premerged['team_score_home']
merged['team_score_away'] = premerged['opponent_team_score_home']

merged['result'] = (premerged['team_score_home'] > premerged['team_score_away']).astype(int)

merged_metrics = pd.merge(merged, premerged, on=['game_id', 'game_date', 'team_home', 'team_away', "rest_days_home", "rest_days_away", "travel_dist_home", "travel_dist_away", "team_score_home", "team_score_away"])
del merged_metrics['home_away_home']
del merged_metrics['home_away_away']
del merged_metrics['home_away_NS_home']
del merged_metrics['home_away_NS_away']
del merged_metrics['attendance_home']
del merged_metrics['attendance_away']
del merged_metrics['opponent_team_score_home']
del merged_metrics['opponent_team_score_away']

merged_metrics['attendance'] = premerged['attendance_home']

In [520]:
merged

,game_id,game_date,team_home,team_away,home_away_NS,rest_days_home,rest_days_away,travel_dist_home,travel_dist_away,team_score_home,team_score_away,result
0,game_2022_43,2021-11-09,chattanooga_mocs,belmont_bruins,1,0.0,0.0,0.0,185.0,70,88,0
1,game_2022_4,2021-11-09,notre_dame_fighting_irish,ohio_bobcats,1,0.0,0.0,0.0,440.0,105,69,1
2,game_2022_5,2021-11-09,boston_college_eagles,harvard_crimson,1,0.0,0.0,0.0,7.0,86,60,1
3,game_2022_6,2021-11-09,florida_state_seminoles,north_florida_ospreys,1,0.0,0.0,0.0,247.0,78,50,1
4,game_2022_7,2021-11-09,central_michigan_chippewas,georgia_tech_yellow_jackets,1,0.0,0.0,0.0,1093.0,40,74,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5214,game_2022_5215,2022-03-13,northern_iowa_panthers,illinois_state_redbirds,0,1.0,1.0,200.0,164.0,48,50,0
5215,game_2022_5219,2022-03-13,drexel_dragons,delaware_blue_hens,1,1.0,1.0,0.0,65.0,59,63,0
5216,game_2022_5216,2022-03-13,american_university_eagles,bucknell_bison,1,3.0,3.0,0.0,230.0,65,54,1
5217,game_2022_5217,2022-03-13,se_louisiana_lady_lions,incarnate_word_cardinals,0,1.0,1.0,525.0,260.0,52,56,0


In [521]:
# Team efficiency
for side in ['home', 'away']:
    merged_metrics[f'FG2%_{side}'] = merged_metrics[f'FGM_2_{side}'] / merged_metrics[f'FGA_2_{side}']
    merged_metrics[f'FG3%_{side}'] = merged_metrics[f'FGM_3_{side}'] / merged_metrics[f'FGA_3_{side}']
    merged_metrics[f'FT%_{side}'] = merged_metrics[f'FTM_{side}'] / merged_metrics[f'FTA_{side}']
    merged_metrics[f'eFG%_{side}'] = (merged_metrics[f'FGM_2_{side}'] + 1.5*merged_metrics[f'FGM_3_{side}']) / \
                            (merged_metrics[f'FGA_2_{side}'] + merged_metrics[f'FGA_3_{side}'])

merged_metrics['Possessions_home'] = 0.96 * (merged_metrics['FGA_2_home'] + merged_metrics['FGA_3_home'] + 
                                    merged_metrics['FTA_home'] * 0.44 - 
                                    merged_metrics['OREB_home'] + merged_metrics['TOV_home'])

merged_metrics['Possessions_away'] = 0.96 * (merged_metrics['FGA_2_away'] + merged_metrics['FGA_3_away'] + 
                                    merged_metrics['FTA_away'] * 0.44 - 
                                    merged_metrics['OREB_away'] + merged_metrics['TOV_away'])

merged_metrics['EFG_diff'] = merged_metrics['eFG%_home'] - merged_metrics['eFG%_away']
merged_metrics['TOV_diff'] = (merged_metrics['TOV_away']/merged_metrics['Possessions_away']) - \
                    (merged_metrics['TOV_home']/merged_metrics['Possessions_home'])
merged_metrics['OREB_diff'] = (merged_metrics['OREB_home']/(merged_metrics['OREB_home'] + merged_metrics['DREB_away'])) - \
                     (merged_metrics['OREB_away']/(merged_metrics['OREB_away'] + merged_metrics['DREB_home']))
merged_metrics['FTR_diff'] = (merged_metrics['FTM_home']/merged_metrics['FGA_2_home']) - \
                    (merged_metrics['FTM_away']/merged_metrics['FGA_2_away'])

In [522]:
# Additional metrics
merged_metrics['Defensive%_home'] = (merged_metrics['BLK_home'] + merged_metrics['STL_home']) / \
                                merged_metrics['Possessions_home']
merged_metrics['Defensive%_away'] = (merged_metrics['BLK_away'] + merged_metrics['STL_away']) / \
                                merged_metrics['Possessions_away']

merged_metrics['AST/TOV_home'] = merged_metrics['AST_home'] / (merged_metrics['TOV_home'] + 1)
merged_metrics['AST/TOV_away'] = merged_metrics['AST_away'] / (merged_metrics['TOV_away'] + 1)

merged_metrics['Travel_Rest_Impact'] = (merged_metrics['travel_dist_away'] / 500) * \
                              (1 / (merged_metrics['rest_days_away'] + 0.1))

merged_metrics['Rest_Days_Diff'] = merged_metrics['rest_days_home'] - merged_metrics['rest_days_away']

In [523]:
merged_metrics

,game_id,game_date,team_home,team_away,home_away_NS,rest_days_home,rest_days_away,travel_dist_home,travel_dist_away,team_score_home,team_score_away,result,FGA_2_home,FGM_2_home,FGA_3_home,FGM_3_home,FTA_home,FTM_home,AST_home,BLK_home,STL_home,TOV_home,TOV_team_home,DREB_home,OREB_home,F_tech_home,F_personal_home,largest_lead_home,notD1_incomplete_home,OT_length_min_tot_home,tz_dif_H_E_home,prev_game_dist_home,FGA_2_away,FGM_2_away,FGA_3_away,FGM_3_away,FTA_away,FTM_away,AST_away,BLK_away,STL_away,TOV_away,TOV_team_away,DREB_away,OREB_away,F_tech_away,F_personal_away,largest_lead_away,notD1_incomplete_away,OT_length_min_tot_away,tz_dif_H_E_away,prev_game_dist_away,attendance,FG2%_home,FG3%_home,FT%_home,eFG%_home,FG2%_away,FG3%_away,FT%_away,eFG%_away,Possessions_home,Possessions_away,EFG_diff,TOV_diff,OREB_diff,FTR_diff,Defensive%_home,Defensive%_away,AST/TOV_home,AST/TOV_away,Travel_Rest_Impact,Rest_Days_Diff
0,game_2022_43,2021-11-09,chattanooga_mocs,belmont_bruins,1,0.0,0.0,0.0,185.0,70,88,0,42,24,17,3,24,13,15,2,1,15,0,21,13,0,19,0.0,False,0.0,0.0,0.0,40,22,30,12,15,8,20,3,11,10,1,21,15,0,26,20.0,False,0.0,0.0,0.0,1041.0,0.571429,0.176471,0.541667,0.483051,0.550000,0.400000,0.533333,0.571429,68.6976,68.7360,-0.088378,-0.072864,-0.034314,0.109524,0.043670,0.203678,0.937500,1.818182,3.700000,0.0
1,game_2022_4,2021-11-09,notre_dame_fighting_irish,ohio_bobcats,1,0.0,0.0,0.0,440.0,105,69,1,46,32,22,9,22,14,31,10,6,12,0,30,13,0,17,38.0,False,0.0,0.0,0.0,48,17,18,7,18,14,6,2,9,19,0,19,16,0,21,3.0,False,0.0,0.0,0.0,0.0,0.695652,0.409091,0.636364,0.669118,0.354167,0.388889,0.777778,0.416667,73.6128,73.8432,0.252451,0.094287,0.058424,0.012681,0.217354,0.148964,2.384615,0.300000,8.800000,0.0
2,game_2022_5,2021-11-09,boston_college_eagles,harvard_crimson,1,0.0,0.0,0.0,7.0,86,60,1,48,27,18,6,17,14,23,7,10,13,0,30,12,0,19,27.0,False,0.0,0.0,0.0,38,12,30,9,13,9,10,2,7,19,1,24,17,0,18,1.0,False,0.0,0.0,0.0,739.0,0.562500,0.333333,0.823529,0.545455,0.315789,0.300000,0.692308,0.375000,71.5008,72.6912,0.170455,0.079563,-0.028369,0.054825,0.237760,0.123811,1.642857,0.500000,0.140000,0.0
3,game_2022_6,2021-11-09,florida_state_seminoles,north_florida_ospreys,1,0.0,0.0,0.0,247.0,78,50,1,51,27,19,4,18,12,16,6,9,17,0,28,20,0,14,31.0,False,0.0,0.0,0.0,35,13,25,6,8,6,5,1,8,23,1,20,12,0,22,0.0,False,0.0,0.0,0.0,1819.0,0.529412,0.210526,0.666667,0.471429,0.371429,0.240000,0.750000,0.366667,71.9232,71.5392,0.104762,0.085139,0.200000,0.063866,0.208556,0.125805,0.888889,0.208333,4.940000,0.0
4,game_2022_7,2021-11-09,central_michigan_chippewas,georgia_tech_yellow_jackets,1,0.0,0.0,0.0,1093.0,40,74,0,27,8,20,4,14,12,7,3,3,15,2,24,7,0,12,0.0,False,0.0,0.0,0.0,52,30,14,2,11,8,21,2,6,5,0,27,13,0,16,35.0,False,0.0,0.0,0.0,0.0,0.296296,0.200000,0.857143,0.297872,0.576923,0.142857,0.727273,0.500000,58.7136,60.3264,-0.202128,-0.172595,-0.145469,0.290598,0.102191,0.132612,0.437500,3.500000,21.860000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5214,game_2022_5215,2022-03-13,northern_iowa_panthers,illinois_state_redbirds,0,1.0,1.0,200.0,164.0,48,50,0,36,14,19,4,10,8,10,5,1,6,0,26,8,0,14,9.0,False,0.0,0.0,0.0,43,18,9,2,12,8,6,2,2,6,0,29,7,0,13,8.0,False,0.0,0.0,0.0,1392.0,0.388889,0.210526,0.800000,0.363636,0.418605,0.222222,0.666667,0.403846,55.1040,54.0288,-0.040210,0.002167,0.004095,0.036176,0.108885,0.074035,1.428571,0.857143,0.298182,0.0
5215,game_2022_5219,2022-03-13,drexel_dragons,delaware_blue_hens,1,1.0,1.0,0.0,65.0,59,63,0,40,16,23,4,17,15,12,0,6,9,0,20,14,0,18,0.0,False,0.0,0.0,0.0,38,14,17,8,19,11,13,2,6,20,1,30,17,0,19,16.0,False,0.0,0.0,0.0,1812.0,0.400000,0.173913,0.882353,0.349206,0.368421,0.470588,0.578947,0.472727,62.8608,63.7056,-0.123521,0.170771,-0.141278,0.085526,0.095449,0.125578,1.200000,0.6

In [ ]:
# Create teams history 2
def create_team_history(df):
    home = df.rename(columns=lambda x: x.replace('_home', '') if '_home' in x else x)
    home = home.rename(columns={'team_home': 'team', 'team_away': 'opponent'})
    home['is_home'] = 1
    
    away = df.rename(columns=lambda x: x.replace('_away', '') if '_away' in x else x)
    away = away.rename(columns={'team_away': 'team', 'team_home': 'opponent'})
    away['is_home'] = 0
    
    return pd.concat([home, away], ignore_index=True)

team_history = create_team_history(merged_metrics)

metrics = [
    'FG2%', 'FG3%', 'FT%', 'eFG%', 'Defensive%', 'AST/TOV',
    'team_score', 'OREB', 'DREB', 'TOV', 'STL', 'BLK'
]

window_size = 10 
team_history = team_history.sort_values(['team', 'game_date'])

for metric in metrics:
    team_history[f'last_{window_size}_{metric}_mean'] = (
        team_history.groupby('team')[metric]
        .shift(1)
        .rolling(window=window_size, min_periods=1)
        .mean()
    )
    
    team_history[f'last_{window_size}_{metric}_trend'] = (
        team_history.groupby('team')[metric]
        .shift(1)
        .rolling(window=window_size, min_periods=1)
        .apply(lambda x: x[-3:].mean() - x[:3].mean() if len(x) >= 3 else 0)
    )

def merge_historical_features(main_df, history_df):
    home_features = history_df.add_prefix('home_team_')
    main_df = pd.merge(
        main_df,
        home_features,
        left_on=['team_home', 'game_date'],
        right_on=['home_team_team', 'home_team_game_date'],
        how='left'
    )
    
    away_features = history_df.add_prefix('away_team_')
    main_df = pd.merge(
        main_df,
        away_features,
        left_on=['team_away', 'game_date'],
        right_on=['away_team_team', 'away_team_game_date'],
        how='left'
    )
    
    return main_df

merged_metrics = merge_historical_features(merged_metrics, team_history)

for metric in metrics:
    merged_metrics[f'hist_{metric}_diff'] = (
        merged_metrics[f'home_team_last_{window_size}_{metric}_mean'] - 
        merged_metrics[f'away_team_last_{window_size}_{metric}_mean']
    )
    
    merged_metrics[f'hist_{metric}_trend_diff'] = (
        merged_metrics[f'home_team_last_{window_size}_{metric}_trend'] - 
        merged_metrics[f'away_team_last_{window_size}_{metric}_trend']
    )

base_features = [
    'rest_days_home', 'rest_days_away',
    'travel_dist_home', 'travel_dist_away',
    'Rest_Days_Diff', 'Travel_Rest_Impact'
]

historical_features = [
    f'hist_{metric}_diff' for metric in metrics
] + [
    f'hist_{metric}_trend_diff' for metric in metrics
]

features = base_features + historical_features
target = 'result'

In [525]:
# Data split
train = merged_metrics[merged_metrics['game_date'] < '2022-02-10']
test = merged_metrics[merged_metrics['game_date'] >= '2022-02-10']

X_train = train[features].fillna(0)
y_train = train[target].clip(0, 1)
X_test = test[features].fillna(0)
y_test = test[target].clip(0, 1)

In [526]:
from sklearn.impute import SimpleImputer
import numpy as np

# Impute missing values with the mean (or any other strategy)
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [527]:
merged_metrics

,game_id,game_date,team_home,team_away,home_away_NS,rest_days_home,rest_days_away,travel_dist_home,travel_dist_away,team_score_home,team_score_away,result,FGA_2_home,FGM_2_home,FGA_3_home,FGM_3_home,FTA_home,FTM_home,AST_home,BLK_home,STL_home,TOV_home,TOV_team_home,DREB_home,OREB_home,F_tech_home,F_personal_home,largest_lead_home,notD1_incomplete_home,OT_length_min_tot_home,tz_dif_H_E_home,prev_game_dist_home,FGA_2_away,FGM_2_away,FGA_3_away,FGM_3_away,FTA_away,FTM_away,AST_away,BLK_away,STL_away,TOV_away,TOV_team_away,DREB_away,OREB_away,F_tech_away,F_personal_away,largest_lead_away,notD1_incomplete_away,OT_length_min_tot_away,tz_dif_H_E_away,prev_game_dist_away,attendance,FG2%_home,FG3%_home,FT%_home,eFG%_home,FG2%_away,FG3%_away,FT%_away,eFG%_away,Possessions_home,Possessions_away,EFG_diff,TOV_diff,OREB_diff,FTR_diff,Defensive%_home,Defensive%_away,AST/TOV_home,AST/TOV_away,Travel_Rest_Impact,Rest_Days_Diff,home_team_game_id,home_team_game_date,home_team_team,home_team_opponent,home_team_home_away_NS,home_team_rest_days,home_team_rest_days_away,home_team_travel_dist,home_team_travel_dist_away,home_team_team_score,home_team_team_score_away,home_team_result,home_team_FGA_2,home_team_FGM_2,home_team_FGA_3,home_team_FGM_3,home_team_FTA,home_team_FTM,home_team_AST,home_team_BLK,home_team_STL,home_team_TOV,home_team_TOV_team,home_team_DREB,home_team_OREB,home_team_F_tech,home_team_F_personal,home_team_largest_lead,home_team_notD1_incomplete,home_team_OT_length_min_tot,home_team_tz_dif_H_E,home_team_prev_game_dist,home_team_FGA_2_away,home_team_FGM_2_away,home_team_FGA_3_away,home_team_FGM_3_away,home_team_FTA_away,home_team_FTM_away,home_team_AST_away,home_team_BLK_away,home_team_STL_away,home_team_TOV_away,home_team_TOV_team_away,home_team_DREB_away,home_team_OREB_away,home_team_F_tech_away,home_team_F_personal_away,home_team_largest_lead_away,home_team_notD1_incomplete_away,home_team_OT_length_min_tot_away,home_team_tz_dif_H_E_away,home_team_prev_game_dist_away,home_team_attendance,home_team_FG2%,home_team_FG3%,home_team_FT%,home_team_eFG%,home_team_FG2%_away,home_team_FG3%_away,home_team_FT%_away,home_team_eFG%_away,home_team_Possessions,home_team_Possessions_away,home_team_EFG_diff,home_team_TOV_diff,home_team_OREB_diff,home_team_FTR_diff,home_team_Defensive%,home_team_Defensive%_away,home_team_AST/TOV,home_team_AST/TOV_away,home_team_Travel_Rest_Impact,home_team_Rest_Days_Diff,home_team_is_home,home_team_home_NS,home_team_rest_days_home,home_team_travel_dist_home,home_team_team_score_home,home_team_FGA_2_home,home_team_FGM_2_home,home_team_FGA_3_home,home_team_FGM_3_home,home_team_FTA_home,home_team_FTM_home,home_team_AST_home,home_team_BLK_home,home_team_STL_home,home_team_TOV_home,home_team_TOV_team_home,home_team_DREB_home,home_team_OREB_home,home_team_F_tech_home,home_team_F_personal_home,home_team_largest_lead_home,home_team_notD1_incomplete_home,home_team_OT_length_min_tot_home,home_team_tz_dif_H_E_home,home_team_prev_game_dist_home,home_team_FG2%_home,home_team_FG3%_home,home_team_FT%_home,home_team_eFG%_home,home_team_Possessions_home,home_team_Defensive%_home,home_team_AST/TOV_home,home_team_last_10_FG2%_mean,home_team_last_10_FG2%_trend,home_team_last_10_FG3%_mean,home_team_last_10_FG3%_trend,home_team_last_10_FT%_mean,home_team_last_10_FT%_trend,home_team_last_10_eFG%_mean,home_team_last_10_eFG%_trend,home_team_last_10_Defensive%_mean,home_team_last_10_Defensive%_trend,home_team_last_10_AST/TOV_mean,home_team_last_10_AST/TOV_trend,home_team_last_10_team_score_mean,home_team_last_10_team_score_trend,home_team_last_10_OREB_mean,home_team_last_10_OREB_trend,home_team_last_10_DREB_mean,home_team_last_10_DREB_trend,home_team_last_10_TOV_mean,home_team_last_10_TOV_trend,home_team_last_10_STL_mean,home_team_last_10_STL_trend,home_team_last_10_BLK_mean,home_team_last_10_BLK_trend,away_team_game_id,away_team_game_date,away_team_team,away_team_opponent,away_team_home_away_NS,away_team_rest_days,away_team_rest_days_away,awa

In [528]:
# 1 model(Logistic Regression)
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(max_iter=50000)
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict_proba(X_test)[:, 1]

In [529]:
# 2 XGBoost
import xgboost as xgb
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    max_depth=50,
    learning_rate=0.1,
    n_estimators=100
)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict_proba(X_test)[:, 1]

In [530]:
# 3 KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=50)
knn_model.fit(X_train, y_train)
knn_preds = knn_model.predict_proba(X_test)[:, 1]

In [531]:
# 4 RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
rf_model = RandomForestClassifier(n_estimators=100, max_depth=50)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict_proba(X_test)[:, 1]

In [532]:
# 5 SVC
from sklearn.svm import SVC
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict_proba(X_test)[:, 1]

In [533]:
# 6 GaussianNB
from sklearn.naive_bayes import GaussianNB
gnb_model = GaussianNB()
gnb_model.fit(X_train, y_train)
gnb_preds = gnb_model.predict_proba(X_test)[:, 1]

In [534]:
# 7 DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier(max_depth=50)
dt_model.fit(X_train, y_train)
dt_preds = dt_model.predict_proba(X_test)[:, 1]

In [535]:
# 8 Ensamble StackingClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.svm import SVC

estimators = [
    ('xgb', xgb.XGBClassifier()),
    ('logreg', LogisticRegression(max_iter=50000)),
    ('gnb', GaussianNB()),
    ('dt', DecisionTreeClassifier(max_depth=50))
]

stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=RandomForestClassifier(),
    cv=5
)
stack_model.fit(X_train, y_train)
stack_preds = stack_model.predict_proba(X_test)[:, 1]

c:\Users\Tora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [536]:
# Evaluate
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
def evaluate_model(name, preds):
    return {
        'Model': name,
        'Accuracy': accuracy_score(y_test, preds > 0.5),
        'ROC AUC': roc_auc_score(y_test, preds),
        'Log Loss': log_loss(y_test, preds)
    }

results = []
for name, preds in [('Logistic Regression', lr_preds),
                    ('XGBoost', xgb_preds),
                    ('KNN', knn_preds),
                    ('Random Forest', rf_preds),
                    ('SVM', svm_preds),
                    ('Gaussian Naive Bayes', gnb_preds),
                    ('Decision Tree', dt_preds),
                    ('Stacking', stack_preds)]:
    results.append(evaluate_model(name, preds))

In [537]:
results

[{'Model': 'Logistic Regression',
  'Accuracy': 0.7050561797752809,
  'ROC AUC': 0.7522095744439143,
  'Log Loss': 0.576773964646084},
 {'Model': 'XGBoost',
  'Accuracy': 0.6903089887640449,
  'ROC AUC': 0.7265518658743388,
  'Log Loss': 0.6784006932251145},
 {'Model': 'KNN',
  'Accuracy': 0.610252808988764,
  'ROC AUC': 0.5865994138095563,
  'Log Loss': 0.6631178947604154},
 {'Model': 'Random Forest',
  'Accuracy': 0.6882022471910112,
  'ROC AUC': 0.7406716610588194,
  'Log Loss': 0.5842653950763484},
 {'Model': 'SVM',
  'Accuracy': 0.5660112359550562,
  'ROC AUC': 0.5941963428160688,
  'Log Loss': 0.69841764929831},
 {'Model': 'Gaussian Naive Bayes',
  'Accuracy': 0.6629213483146067,
  'ROC AUC': 0.7296377310009645,
  'Log Loss': 0.7448578706725153},
 {'Model': 'Decision Tree',
  'Accuracy': 0.6060393258426966,
  'ROC AUC': 0.5941911790524701,
  'Log Loss': 14.199781988268768},
 {'Model': 'Stacking',
  'Accuracy': 0.6573033707865169,
  'ROC AUC': 0.6887056097062232,
  'Log Loss': 0.6

In [538]:
X_train = train[features].fillna(0).to_numpy()  # Convert to numpy array
y_train = train[target].clip(0, 1).to_numpy()
X_test = test[features].fillna(0).to_numpy()
y_test = test[target].clip(0, 1).to_numpy()

models = {
    'Logistic Regression': LogisticRegression(max_iter=50000),
    'XGBoost': xgb.XGBClassifier(
        objective='binary:logistic',
        max_depth=5,  # Reduced from 50 to prevent overfitting
        learning_rate=0.1,
        n_estimators=100
    ),
    'KNN': KNeighborsClassifier(n_neighbors=50),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=10),  # Reduced depth
    'SVM': SVC(probability=True, kernel='rbf', C=1.0),
    'Gaussian Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(max_depth=5)  # Reduced depth
}

# Train and validate individual models
base_predictions = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    val_preds = model.predict_proba(X_test)[:, 1]
    base_predictions[name] = val_preds
    print(f"{name} Validation ROC AUC: {roc_auc_score(y_test, val_preds):.4f}")

# 2. Build Proper Stacking Ensemble
stacking_base_models = [
    ('xgb', models['XGBoost']),
    ('logreg', models['Logistic Regression']),
    ('rf', models['Random Forest']),
    ('svm', models['SVM'])
]

# Use logistic regression as meta-learner for better interpretability
stack_model = StackingClassifier(
    estimators=stacking_base_models,
    final_estimator=LogisticRegression(max_iter=5000),
    cv=5,
    passthrough=False  # Prevent original features from leaking into meta-model
)

# Train stacking on training data
stack_model.fit(X_train, y_train)

# 3. Final Evaluation on Test Set
results = []

# Evaluate base models on test set
for name, model in models.items():
    test_preds = model.predict_proba(X_test)[:, 1]
    results.append(evaluate_model(name, test_preds))

# Evaluate stacking on test set
stack_test_preds = stack_model.predict_proba(X_test)[:, 1]
results.append(evaluate_model('Stacking', stack_test_preds))

# Create results DataFrame
results_df = pd.DataFrame(results)
print(results_df.sort_values('ROC AUC', ascending=False))


Logistic Regression Validation ROC AUC: 0.7522
XGBoost Validation ROC AUC: 0.7303
KNN Validation ROC AUC: 0.5866
Random Forest Validation ROC AUC: 0.7332
SVM Validation ROC AUC: 0.5942
Gaussian Naive Bayes Validation ROC AUC: 0.7296
Decision Tree Validation ROC AUC: 0.6836


c:\Users\Tora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                  Model  Accuracy   ROC AUC  Log Loss
0   Logistic Regression  0.705056  0.752210  0.576774
3         Random Forest  0.683287  0.733211  0.586682
7              Stacking  0.687500  0.732021  0.588239
1               XGBoost  0.675562  0.730330  0.589469
5  Gaussian Naive Bayes  0.662921  0.729638  0.744858
6         Decision Tree  0.633427  0.683609  0.669707
4                   SVM  0.566713  0.594239  0.696497
2                   KNN  0.610253  0.586599  0.663118


In [539]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

X_train = train[features].fillna(0)
y_train = train[target].clip(0, 1)
X_test = test[features].fillna(0)
y_test = test[target].clip(0, 1)


preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', X_train.columns)
    ],
    verbose_feature_names_out=False
)

pipeline = make_pipeline(
    preprocessor,
    GaussianNB()
)

pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)  # No warnings
print(evaluate_model("Transformer", preds))


{'Model': 'Transformer', 'Accuracy': 0.6629213483146067, 'ROC AUC': 0.6673193663855513, 'Log Loss': 12.149546086219265}


In [540]:
import torch
from torch.nn import functional as F
from torch import nn, einsum
from einops import rearrange

In [541]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, 
                         batch_first=True, dropout=0.2)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = x.unsqueeze(1)
        gru_out, _ = self.gru(x)
        return self.fc(gru_out[:, -1, :])

In [542]:
# 4 Ensemble Model
class EnsembleModel(nn.Module):
    def __init__(self, models):
        super().__init__()
        self.models = nn.ModuleList(models)
        self.meta = nn.Linear(len(models), 1)
        
    def forward(self, x):
        outputs = [model(x) for model in self.models]
        stacked = torch.stack(outputs, dim=1)
        return torch.sigmoid(self.meta(stacked))

In [543]:
# 5 March Madness Model
class MarchMadnessModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.tourney_features = nn.Sequential(
            nn.Linear(5, 16),
            nn.BatchNorm1d(16),
            nn.GELU(),
            nn.Dropout(0.4)
        )
        self.general_features = nn.Sequential(
            nn.Linear(input_size-5, 64),
            nn.LayerNorm(64),
            nn.SiLU()
        )
        self.combined = nn.Sequential(
            nn.Linear(80, 40),
            nn.BatchNorm1d(40),
            nn.LeakyReLU(0.2),
            nn.Linear(40, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        tourney_feats = self.tourney_features(x[:, :5])
        general_feats = self.general_features(x[:, 5:])
        combined = torch.cat([tourney_feats, general_feats], dim=1)
        return self.combined(combined)

In [544]:
# 6 Feature Augmented Model
class FeatureAugmentedModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.ast_tov = nn.Sequential(
            nn.Linear(4, 8),
            nn.BatchNorm1d(8),
            nn.GELU()
        )
        self.defense = nn.Sequential(
            nn.Linear(2, 4),
            nn.LayerNorm(4),
            nn.SiLU()
        )
        self.main = nn.Sequential(
            nn.Linear(input_size+8+4, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        ast_tov_feats = self.ast_tov(x[:, [features.index('AST/TOV_home'), 
                                         features.index('AST/TOV_away')]])
        defense_feats = self.defense(x[:, [features.index('Defensive%_home'),
                                         features.index('Defensive%_away')]])
        combined = torch.cat([x, ast_tov_feats, defense_feats], dim=1)
        return self.main(combined)

In [545]:
# 7 LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                           batch_first=True, dropout=0.2)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = x.unsqueeze(1)
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])

In [546]:
# 8 RNN
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers,
                         batch_first=True, nonlinearity='relu', dropout=0.2)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = x.unsqueeze(1)
        rnn_out, _ = self.rnn(x)
        return self.fc(rnn_out[:, -1, :])

In [547]:
class ModelTrainer:
    def __init__(self, model_class, device='cuda'):
        self.device = torch.device(device)
        self.model = model_class.to(self.device)
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=1e-3)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min')
        self.criterion = nn.BCEWithLogitsLoss()
    
    def train_epoch(self, loader):
        self.model.train()
        total_loss = 0
        for inputs, targets in loader:
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(inputs).squeeze()
            loss = self.criterion(outputs, targets)
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(loader)
    
    def evaluate(self, loader):
        self.model.eval()
        total_loss = 0
        preds, true = [], []
        with torch.no_grad():
            for inputs, targets in loader:
                inputs = inputs.to(self.device)
                outputs = self.model(inputs).cpu().squeeze()
                loss = self.criterion(outputs, targets)
                total_loss += loss.item()
                preds.append(outputs.numpy())
                true.append(targets.numpy())
        return total_loss/len(loader), np.concatenate(preds), np.concatenate(true)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

X_train = train[features].fillna(0)
y_train = train[target].clip(0, 1)
X_test = test[features].fillna(0)
y_test = test[target].clip(0, 1)

X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.FloatTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test.values)
y_test_tensor = torch.FloatTensor(y_test.values)

input_size = len(features)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

models = {
    'GRU': GRUModel(input_size),
    'LSTM': LSTMModel(input_size),
    'RNN': RNNModel(input_size),
    'MarchMadness': MarchMadnessModel(input_size),
}

# Convert data to PyTorch datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Train all models
results = {}
predictions = {}

for name, model_class in models.items():
    print(f"\nTraining {name}...")

    trainer = ModelTrainer(model_class, device)
    
    best_loss = float('inf')
    no_improve = 0
    for epoch in range(100):
        train_loss = trainer.train_epoch(train_loader)
        val_loss, val_preds, val_true = trainer.evaluate(test_loader)
        
        trainer.scheduler.step(val_loss)

        if epoch == 0:
            torch.save(trainer.model.state_dict(), f'best_{name}.pth')
        
        if val_loss < best_loss:
            best_loss = val_loss
            no_improve = 0
            torch.save(trainer.model.state_dict(), f'best_{name}.pth')
        else:
            no_improve += 1
            if no_improve >= 5:
                break
    
    # Load best model
    trainer.model.load_state_dict(torch.load(f'best_{name}.pth'))
    _, test_preds, test_true = trainer.evaluate(test_loader)
    predictions[name] = test_preds
    results[name] = evaluate_model(name, test_preds)

# %% [code]
# --------------------------
# 7. Ensemble & Final Evaluation
# --------------------------
# Create ensemble dataset
ensemble_train = np.stack([preds for preds in predictions.values()], axis=1)
ensemble_test = np.stack([preds for preds in predictions.values()], axis=1)

# Train meta-learner
meta_model = xgb.XGBClassifier(
    objective='binary:logistic',
    max_depth=3,
    learning_rate=0.05,
    n_estimators=50
)
meta_model.fit(ensemble_train, test_true)

# Evaluate ensemble
ensemble_preds = meta_model.predict_proba(ensemble_test)[:, 1]
results['Ensemble'] = evaluate_model('Meta-Ensemble', ensemble_preds)

# Print final results
results_df = pd.DataFrame(results.values())
print(results_df.sort_values('ROC AUC', ascending=False))


Training GRU...

Training LSTM...

Training RNN...

Training MarchMadness...
           Model  Accuracy   ROC AUC  Log Loss
4  Meta-Ensemble  0.705056  0.776765  0.555235
3   MarchMadness  0.669242  0.733066  1.097989
0            GRU  0.621489  0.712748  1.472751
1           LSTM  0.636938  0.705406  1.354283
2            RNN  0.613764  0.671922  2.012486
